[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gee-community/geemap/blob/master/docs/workshops/Alaska_2024_Part3.ipynb)

**Geospatial Cloud Computing with the GEE Python API - Part 3**

-   Notebook: <https://geemap.org/workshops/Alaska_2024_Part3>
-   Earth Engine: <https://earthengine.google.com>
-   Geemap: <https://geemap.org>

## Introduction

This notebook contains the materials for the third part of the workshop **Geospatial Cloud Computing with the GEE Python API** at the University of Alaska Fairbanks.

This workshop provides an introduction to cloud-based geospatial analysis using the Earth Engine Python API. Attendees will learn the basics of Earth Engine data types and how to visualize, analyze, and export Earth Engine data in a Jupyter environment with geemap. In addition, attendees will learn how to develop and deploy interactive Earth Engine web apps with Python. Through practical examples and hands-on exercises, attendees will enhance their learning experience. During each hands-on session, attendees will walk through Jupyter Notebook examples on Google Colab with the instructors. At the end of each session, they will complete a hands-on exercise to apply the knowledge they have learned.

### Agenda

The workshop is divided into three parts. The third part will cover the following topics:

* Image Classification (focused on land cover in Alaska)
* Accuracy assessment
* Create and export maps 
* Building interactive web apps

### Prerequisites

-   To use geemap and the Earth Engine Python API, you must [register](https://code.earthengine.google.com/register) for an Earth Engine account and follow the instructions [here](https://docs.google.com/document/d/1ZGSmrNm6_baqd8CHt33kIBWOlvkh-HLr46bODgJN1h0/edit?usp=sharing) to create a Cloud Project. Earth Engine is free for [noncommercial and research use](https://earthengine.google.com/noncommercial). To test whether you can use authenticate the Earth Engine Python API, please run [this notebook](https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/notebooks/geemap_colab.ipynb) on Google Colab.